In [1]:
import matplotlib.pyplot as plt

from mortgage import Loan
from dataclasses import dataclass

In [439]:
@dataclass
class percents:
    capex:float = 0.1
    misc_ex:float = 0.05
    repairs:float = 0.08
    mgmt:float = 0.1
    vacancy:float = 0.05
    

@dataclass
class utilities:
    """monthly value estimations, rough estimations"""
    water_sewer:float = 120
    gas:float = 120
    electric:float = 120
    garbage:float = 40

@dataclass
class webvalues:
    taxes:float=100
    insurance:float=100
    hoa:float = 0
    lawn:float = 50
    
class RealEstate(percents, utilities, webvalues):
    def __init__(self, property_type, list_price, rental_income=0, webvalues=webvalues, utilities=utilities, percents=percents):
        '''Calculate monthly expenses'''
        
        if property_type not in ["sfh", "multi"]:
            assert False
        
        # protected attributes, these stay set constant
        self._property_type = property_type
        self._rental_income = rental_income
        self._list_price = list_price
        self.taxes = webvalues.taxes
        self.insurance = webvalues.insurance
        self.hoa = webvalues.hoa
        self.lawn = webvalues.lawn
        
        self.property_type = self._property_type
        self.rental_income = self._rental_income
        self.list_price = self._list_price
        self.set_expenses()

    def compute_payment(self):
        self.down_payment = self.list_price * self.down_payment_pct
        self.principal_amount = self.list_price - self.down_payment
        self.monthly_payment = float(Loan(principal=self.principal_amount, interest=0.05, term=30).monthly_payment)

    def set_expenses(self):
        if self.property_type == "multi":
            self.down_payment_pct = 0.2
            self.compute_payment()
            
            try:
                assert self.rental_income != 0
            except AssertionError:
                print("Warning: Rental property income is 0, this isn't good.")
            
            self.best_case = True
 
            self.capex = self.rental_income * percents.capex
            self.mgmt_fees = self.rental_income * percents.mgmt
            self.repairs = self.rental_income * percents.repairs
            self.misc_expenses = self.rental_income * percents.misc_ex
            self.vacancy = self.rental_income * percents.vacancy
        
        
            self.water_sewer = 0
            self.garbage = 0
            self.electric = 0
            self.gas = 0
            
        else:
            # personal property, variable expences will be a function of mortgage
            self.down_payment_pct = 0.03
            self.compute_payment()
            
            self.capex = self.monthly_payment * percents.capex 
            self.mgmt_fees = 0
            self.misc_expenses = self.monthly_payment * percents.misc_ex
            self.repairs = self.monthly_payment * percents.repairs
            self.vacancy = 0
            
            self.water_sewer = utilities.water_sewer
            self.garbage = utilities.garbage
            self.electric = utilities.electric
            self.gas = utilities.gas

            
    def get_rent(self):
        return self.rental_income
    
    @property
    def _expenses(self):
        return [self.capex, 
                self.mgmt_fees, 
                self.misc_expenses, 
                self.repairs, 
                self.vacancy, 
                self.water_sewer,
                self.garbage,
                self.lawn,
                self.electric,
                self.gas,
                self.hoa,
                self.taxes,
                self.insurance,
                self.monthly_payment
               ]
    
    def monthly_expenses(self):
        return sum(self._expenses)
    
    def cashflow(self):
        if self.property_type == "multi":
            return round(float(self.rental_income - self.monthly_expenses()), 2)
        else:
            return round(float(0 - self.monthly_expenses()), 2)
    
    def effective_rent(self):
        """assuming i live here, create a wrapper for that"""
        self.property_type = "sfh"
        self.set_expenses()
        cashflow = self.cashflow()
        self.reset_values()
        return cashflow
    
    def roi_as_pct(self):
        return round((self.cashflow() * 12) / self.down_payment, 2) * 100
    
    def print_numbers(self):
        print(f"values for property as a {self.property_type} property")
        print(f"expenses: ${self.monthly_expenses()} / month")
        print(f"cashflow: ${self.cashflow()} / month")
        print(f"down_payment: ${self.down_payment} down")
        print(f"return on investment: {self.roi_as_pct()} %")
        print(f"time to recoup investment: {self.time_to_recoup()}")
        
        
    def analyze(self):
        self.property_type = "multi"
        self.set_expenses()
        self.print_numbers()
        
        print("------------")
        self.property_type = "sfh"
        self.set_expenses()
        self.print_numbers()
    
        self.reset_values()
        
    def reset_values(self):
        self.property_type = self._property_type
        self.list_price = self._list_price
        self.rental_income = self._rental_income
        self.set_expenses()
        print("resetting back to initialized values")
        
    def time_to_recoup(self):
        """compute the total time before I gain my investment back in months"""
        months = self.down_payment / abs(self.cashflow())
        if months > 12:
            return (months / 12, "years")
        else:
            return (months, "months")
    
    def vizualize_property_value(self):
        pass
    
    def generate_full_report(self):
        """this should be the verbose output where assumptions are listed so user knows what to watch out for"""

In [434]:
## numners unlikely to change
interest_rate = .05
num_months = 360


In [452]:
## enter the numbers that need to be changed here
list_price = 269000

# grab from webpage
taxes = 511
insurance=100
hoa_fees = 0
rental_income = 57000/12

In [453]:
# utils = utilValues(gas = 100)
# print(utils.gas)
# if i pass in utils, then i can modify the RealEstate class without changing the internal code
# but if i dont, then my class will be populated with some defauly values

# next steps
# analyze as ...
# exclusive airbnb (utilities paid by me)
# pure single family live in
# if its Multifamily, what are the numbers for 


wv = webvalues(insurance=insurance, taxes=taxes, hoa=hoa_fees)

house = RealEstate(property_type="sfh", rental_income=rental_income, webvalues=wv, list_price=list_price)
house.analyze()

values for property as a multi property
expenses: $3621.24 / month
cashflow: $1128.76 / month
down_payment: $53800.0 down
return on investment: 25.0 %
time to recoup investment: (3.9719101787211923, 'years')
------------
values for property as a sfh property
expenses: $2783.8979 / month
cashflow: $-2783.9 / month
down_payment: $8070.0 down
return on investment: -413.99999999999994 %
time to recoup investment: (2.898811020510794, 'months')
resetting back to initialized values


In [421]:
rent.remainder

<bound method RentRoll.remainder of RentRoll(capex_pct=0.1, misc_ex_pct=0.05, mgmt_pct=0.1)>